In [1]:

import pandas as pd
import os

In [8]:

def generate_ddl_from_file(file_path):
    """
    Generate SQL DDL from a CSV or Excel file
    
    Args:
        file_path (str): Path to the input file (.csv, .xls, or .xlsx)
    
    Returns:
        str: SQL DDL statement
    """
    
    # Extract the file extension using os.path.splitext
    _, ext = os.path.splitext(file_path)
    file_extension = ext.lower()[1:]  # Remove the leading dot

    # Check if the file extension is valid
    if file_extension == 'csv':
        df = pd.read_csv(file_path)
    elif file_extension in ['xls', 'xlsx']:
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use .csv, .xls, or .xlsx files")

    dtype_mapping = {
        'object': 'TEXT',
        'int64': 'INTEGER',
        'float64': 'NUMERIC',
        'datetime64[ns]': 'TIMESTAMP',
        'bool': 'BOOLEAN'
    }
    
    # Use os.path.basename and os.path.splitext to extract the table name
    table_name = os.path.splitext(os.path.basename(file_path))[0].lower()
    columns = []
    for column, dtype in df.dtypes.items():
        sql_type = dtype_mapping.get(str(dtype), 'TEXT')
        columns.append(f'    "{column}" {sql_type}')
    
    # Data table creation DDL
    ddl = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    ddl += ",\n".join(columns)
    ddl += "\n);"
    
    # Data insertion DDL / Extraction step of the ETL process
    columns_list = ", ".join([f'"{col}"' for col in df.columns])
    insert_statements = []
    for index, row in df.iterrows():
        values = []
        for col in df.columns:
            if pd.isna(row[col]):
                values.append('NULL')
            elif isinstance(row[col], (int, float)):
                values.append(str(row[col]))
            elif isinstance(row[col], bool):
                values.append('TRUE' if row[col] else 'FALSE')
            elif isinstance(row[col], str):
                values.append(f"'{row[col].replace('\'', '\'\'')}'")
            else:
                values.append(str(row[col]))
        values_str = ", ".join(values)
        insert_statements.append(f"INSERT INTO {table_name} ({columns_list}) VALUES ({values_str});")
    ddl += "\n\n" + "\n".join(insert_statements)
    ddl += "\n"

    return df, ddl


In [10]:
# Directory containing the data files
data_dir = './Data_Selected'
output_dir = './sql_scripts/ddl-dml'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through every file in the data directory
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    
    # Process the file using your SQL DDL function
    try:
        _, sql_script = generate_ddl_from_file(file_path)
    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        continue
    
    # Build the output file name based on the original file's name
    base_name, _ = os.path.splitext(file_name)
    output_file = os.path.join(output_dir, base_name + '.sql')
    
    # Write the generated SQL script to the output file
    with open(output_file, 'w') as f:
        f.write(sql_script)
    
    print(f"SQL script for {file_name} saved to {output_file}")


Error processing Ar76c.txt: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing Ar76f.txt: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR79C.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR79F.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR80C.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR80F.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR83C.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR83F.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR85C.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR85F.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files
Error processing AR87C.TXT: Unsupported file format. Please use .csv, .xls, or .xlsx files

### cells below are explicitly for testing / debugging purposes

In [ ]:
_, test_sql_ddl = generate_ddl_from_file('../OriginalMAIdata/ar_1999.xls')
print(test_sql_ddl)
ddl_path = './sql_scripts/ddl_script.sql'
with open(ddl_path, 'w') as f:
    f.write(test_sql_ddl)


CREATE TABLE IF NOT EXISTS ar_1999 (
    "CODIGO" INTEGER,
    "DIST_CONC_FREG" TEXT,
    "T_ELEI" TEXT,
    "D_ELEI" TEXT,
    "SIGLA" TEXT,
    "T_DADOS" TEXT,
    "VOTOS" INTEGER,
    "P_VOTOS" NUMERIC,
    "MANDATOS" INTEGER
);

INSERT INTO ar_1999 ("CODIGO", "DIST_CONC_FREG", "T_ELEI", "D_ELEI", "SIGLA", "T_DADOS", "VOTOS", "P_VOTOS", "MANDATOS") VALUES (10000, 'AVEIRO', 'ar', '10/10/1999', 'CDS/PP', 'O', 49196, 13.579999923706055, 2);
INSERT INTO ar_1999 ("CODIGO", "DIST_CONC_FREG", "T_ELEI", "D_ELEI", "SIGLA", "T_DADOS", "VOTOS", "P_VOTOS", "MANDATOS") VALUES (10000, 'AVEIRO', 'ar', '10/10/1999', 'B.E.', 'O', 4677, 1.2899999618530273, 0);
INSERT INTO ar_1999 ("CODIGO", "DIST_CONC_FREG", "T_ELEI", "D_ELEI", "SIGLA", "T_DADOS", "VOTOS", "P_VOTOS", "MANDATOS") VALUES (10000, 'AVEIRO', 'ar', '10/10/1999', 'MPT', 'O', 842, 0.23000000417232513, 0);
INSERT INTO ar_1999 ("CODIGO", "DIST_CONC_FREG", "T_ELEI", "D_ELEI", "SIGLA", "T_DADOS", "VOTOS", "P_VOTOS", "MANDATOS") VALUES (10000, 'A